In [ ]:
import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use(style='ggplot')
plt.rcParams['figure.figsize'] = (10, 6)

import seaborn as sns

from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [ ]:
!apt-get install p7zip
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/train.tsv.7z
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/test.tsv.7z
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/sample_submission.csv.7z

In [ ]:
!unzip /kaggle/input/mercari-price-suggestion-challenge/sample_submission_stg2.csv.zip
!unzip /kaggle/input/mercari-price-suggestion-challenge/test_stg2.tsv.zip

>  # Workflow
1. <a href="#Mô-tả-bài-toán"> Mô tả bài toán</a> 
2. <a href="#Xem-xét-và-khái-quát-dữ-liệu"> Xem xét và khái quát dữ liệu</a>
3. <a href="#Kiểm-tra-phân-phối-dữ-liệu"> Kiểm tra phân phối dữ liệu</a>
4. <a href="#Xử-lý-dữ-liệu"> Xử lý dữ liêu</a>
5. <a href="#Đánh-giá"> Đánh giá</a>
6. <a href="#Huấn-luyện"> Huấn Luyện</a>
7. <a href="#Kết-quả"> Kết quả</a>

# Mô tả bài toán
Xây dựng một thuật toán tự động đề xuất giá sản phẩm phù hợp khi đã được cung cấp các mô tả văn bản do người dùng nhập về sản phẩm của Mercari, bao gồm các chi tiết như tên danh mục sản phẩm, tên thương hiệu và tình trạng mặt hàng.

Input: Dữ liệu dạng text gồm **name**, **item_condition**, **category**, **brand_name**, **shipping** và **item_description** về sản phẩm.

Output: Giá của sản phẩm.

# Xem xét và khái quát dữ liệu
**Tại đây ta sẽ phân tích và xem xét dữ liệu được cung cấp sau đó đưa ra phương pháp và xử lý dữ liệu**

    Đầu tiên xem xét số hàng và cột của tập train và test.

In [ ]:
train = pd.read_csv('train.tsv', sep = '\t')
test = pd.read_csv('test_stg2.tsv', sep='\t')
print ("Train data shape:", train.shape)
print ("Test data shape:", test.shape)

**Nhận xét :**
*     Tệp trai.tsv gồm *14822535* dòng và 8 cột.
*     Tệp test.tsv gồm *3460725* dòng và 7 cột.

**Vì tập train là tập ta cần xử lý để huấn luyện nên ta phân tích tập train.**

    Ta xem xét các hàng đầu tiên của tập train

In [ ]:
train.head()

**Nhận xét :**
*   Tập train có các cột bao gồm name,item_condition_id,category_name,brand_name ,shipping,item_description và price

**Sau đó ta tiếp tục xem thông tin chi tiết về kiểu dữ liệu trong tập train**

In [ ]:
train.info()

Nhận thấy cột Category_name, brand_name and item_description chứa các giá trị null

--> **Cần thực hiện chuẩn hóa các giá trị null thành các giá trị có thể xử lý**

# Kiểm tra phân phối dữ liệu

**Price**

Kiểm tra phân phối của biến Price

In [ ]:
train.price.describe().apply(lambda x: format(x, 'f'))

Ta thấy có vẻ như phân bố của biến Price lệch, biểu đồ hóa biến Price để kiểm tra

In [ ]:
sns.distplot(train['price'])

Phân bố lệch về bên trái. Ta sử dụng hàm đồng biến log() sẽ đưa đồ thị giá về gần dạng phân bố chuẩn.

In [ ]:
train['price'] = np.log1p(train['price'])
sns.distplot(train['price'])

Oke, Giờ thì phân bố đã có dạng phân bố chuẩn -> bài toán hồi quy

**Ta tiếp tục kiểm tra phân bố các biến còn lại :**

**Shipping**

In [ ]:
train['shipping'].value_counts()

In [ ]:
train.shipping.value_counts()/len(train)

Số lượng mặt hàng không shipping chiếm 55% và 45% còn lại được shipping

**Category**

Kiểm tra số lượng giá trị category khác nhau trên tập train :

In [ ]:
print(train['category_name'].nunique())

Xem xét 10 giá trị đầu của category

In [ ]:
train['category_name'].value_counts().head(10)

In [ ]:
train['category_name'].isnull().sum()

Nhận thấy các giá trị trong cột category tạo nên từ 3 category riêng

Có 6327 giá trị null cho cột category

==> Từ 2 nhận xét trên ta đề xuất tách category thành 3 cột **main_cat**,**sub_cat** và **item_cat** và chuẩn hóa giá trị **null** để dễ phân tích

**Item_description**

Kiểm tra phân phối biến item_description

In [ ]:
train['item_description'].value_counts().head()

**Item_condition_id**

Kiểm tra danh sách giá trị biến item_condition_id

In [ ]:
train['item_condition_id'].value_counts()

**Brand_name**

Kiểm tra danh sách giá trị biến brand_name

In [ ]:
train['brand_name'].value_counts().head()

# Xử lý dữ liệu
## Làm sạch dữ liệu
**Xử lý cột Category_name**

In [ ]:
def category_split(category_name):
    try:
        return category_name.split('/')
    except:
        return ['Missing', 'Missing', 'Missing']
train['main_cat'], train['sub_cat'], train['item_cat'] = zip(*train['category_name'].apply(lambda x: category_split(x)))
test['main_cat'], test['sub_cat'], test['item_cat'] = zip(*test['category_name'].apply(lambda x: category_split(x)))

**Xử lý dữ liệu chứa giá trị null**

Thay thế giá trị null thành giá trị Missing trên 3 cột: **category**, **brand** và **item_description**

In [ ]:
train['category_name'] = train['category_name'].fillna(value='Missing')
train['brand_name'] = train['brand_name'].fillna(value='Missing')
train['item_description'] = train['item_description'].fillna(value='Missing')

test['category_name'] = test['category_name'].fillna(value='Missing')
test['brand_name'] = test['brand_name'].fillna(value='Missing')
test['item_description'] = test['item_description'].fillna(value='Missing')

**Kiểm tra lại dữ liệu đã làm sạch qua các bước đã qua trên tập train :**

In [ ]:
train.head()

In [ ]:
train.nunique()

Như ta thấy số loại giá trị trên dữ liệu của **name** và **item_description** lớn hơn hẳn các columns khác nên:

**==> Vectơ hóa name và item_description**

**==> Phân loại theo label các features còn lại**

## Vectơ hóa dữ liệu

**Vectơ hóa Name column**

Để thực hiện vectơ hóa cột name ta dùng mô hình BoW (Bag-of-words)

Mô hình bag-of-words là một mô hình đơn giản hóa được sử dụng trong xử lý ngôn ngữ tự nhiên và truy xuất thông tin (IR). Trong mô hình này, một văn bản (chẳng hạn như một câu hoặc một tài liệu) được biểu thị như một túi (nhiều tập hợp) các từ của nó, không tính đến ngữ pháp và thậm chí cả trật tự từ nhưng vẫn giữ tính đa nghĩa

Tham khảo : https://en.wikipedia.org/wiki/Bag-of-words_model

In [ ]:
cnt_vec = CountVectorizer()

X_train_name = cnt_vec.fit_transform(train['name'])
X_test_name = cnt_vec.transform(test['name'])

In [ ]:
print(X_train_name.shape)
print(X_test_name.shape)

**Vectơ hóa Item_Description Column**

Để thực hiện vectơ hóa cột name ta dùng mô hình TF-IDF(Term Frequency – Inverse Document Frequency)

TF-IDF là 1 kĩ thuật sử dụng trong khai phá dữ liệu văn bản. Trọng số này được sử dụng để đánh giá tầm quan trọng của một từ trong một văn bản. Giá trị cao thể hiện độ quan trọng cao và nó phụ thuộc vào số lần từ xuất hiện trong văn bản nhưng bù lại bởi tần suất của từ đó trong tập dữ liệu

Tham khảo : https://vi.wikipedia.org/wiki/Tf%E2%80%93idf

In [ ]:
tfidf_descp = TfidfVectorizer(max_features=50000, ngram_range=(1, 3), stop_words='english')

X_train_descp = tfidf_descp.fit_transform(train['item_description'])
X_test_descp = tfidf_descp.transform(test['item_description'])

In [ ]:
train.head()

**Phân loại các features còn lại thông qua LabelBinarizer**

LabelBinarizer là một thuật toán phân loại hồi quy và nhị phân có sẵn trong scikit. Nó giúp ta phân loại dữ liệu theo các label theo kiểu one-vs-all

Tham khảo : https://www.kite.com/python/docs/sklearn.preprocessing.LabelBinarizer

In [ ]:
from sklearn.preprocessing import LabelBinarizer

Tại đây ta sẽ áp dụng labelBinarizer trên tất cả các biến còn lại trên tập dữ liệu huấn luyện và kiểm tra.

In [ ]:
lb_brand_name = LabelBinarizer(sparse_output=True)
X_train_brand = lb_brand_name.fit_transform(train['brand_name'])
X_test_brand = lb_brand_name.transform(test['brand_name'])

lb_item_cond_id = LabelBinarizer(sparse_output=True)
X_train_item_condition_id = lb_item_cond_id.fit_transform(train['item_condition_id'])
X_test_item_condition_id = lb_item_cond_id.transform(test['item_condition_id'])

lb_shipping = LabelBinarizer(sparse_output=True)
X_train_shipping = lb_shipping.fit_transform(train['shipping'])
X_test_shipping = lb_shipping.transform(test['shipping'])

In [ ]:
lb_main_cat = LabelBinarizer(sparse_output=True)
X_train_main_cat = lb_main_cat.fit_transform(train['main_cat'])
X_test_main_cat = lb_main_cat.transform(test['main_cat'])

lb_sub_cat = LabelBinarizer(sparse_output=True)
X_train_sub_cat = lb_sub_cat.fit_transform(train['sub_cat'])
X_test_sub_cat = lb_sub_cat.transform(test['sub_cat'])

lb_item_cat = LabelBinarizer(sparse_output=True)
X_train_item_cat = lb_item_cat.fit_transform(train['item_cat'])
X_test_item_cat = lb_item_cat.transform(test['item_cat'])

In [ ]:
# Full dataframe printing
print(type(X_train_brand), type(X_train_item_condition_id), type(X_train_shipping))
print(type(X_test_brand), type(X_test_item_condition_id), type(X_test_shipping))

In [ ]:
# Train dataframe printing
print('X_train_brand shape:', X_train_brand.shape)
print('X_train_item_cond_id shape:', X_train_item_condition_id.shape)
print('X_train_shipping shape:', X_train_shipping.shape)
print('X_train_main_cat shape:', X_train_main_cat.shape)
print('X_train_sub_cat shape:', X_train_sub_cat.shape)
print('X_train_item_cat shape:', X_train_item_cat.shape)

**==> Gán nhãn hoàn thành trên tập train**

In [ ]:
# Test dataframe printing
print('X_test_brand shape:', X_test_brand.shape)
print('X_test_item_cond_id shape:', X_test_item_condition_id.shape)
print('X_test_shipping shape:', X_test_shipping.shape)
print('X_test_main_cat shape:', X_test_main_cat.shape)
print('X_test_sub_cat shape:', X_test_sub_cat.shape)
print('X_test_item_cat shape:', X_test_item_cat.shape)

**==> Gán nhãn hoàn thành trên tập test**

## Xếp chồng X_train

Xếp chồng các khối dữ liệu đã xử lý thành ma trận X_train

In [ ]:
from scipy.sparse import hstack
import gc

In [ ]:
sparse_matrix_list = (X_train_name, X_train_descp, X_train_brand, 
                      X_train_item_condition_id, X_train_shipping, 
                      X_train_main_cat, X_train_sub_cat, X_train_item_cat)

Chuyển đổi ma trận này thành dạng dạng hàng được nén để sử dụng

In [ ]:
X_train = hstack(sparse_matrix_list).tocsr()
print(type(X_train), X_train.shape)

In [ ]:
del X_train
gc.collect()

# Đánh giá

Hàm evaluate có dạng như sau:

In [ ]:
def rmsle(y, y_pred):
    return np.sqrt(np.mean(np.power(np.log1p(y) - np.log1p(y_pred), 2)))

def evaluate_orig_price(y_test, preds):
    preds_exmpm = np.expm1(preds)
    y_test_exmpm = np.expm1(y_test)
    
    return rmsle(y_test_exmpm, preds_exmpm)

Gói các features phục vụ quá trình training và testing.

Để đánh giá độ hiệu quả của model, ta lấy 20% tập train làm tập valid.

In [ ]:
def model_train_predict(model, matrix_list):
    X = hstack(matrix_list).tocsr()
    X_train, X_test, y_train, y_test = train_test_split(X, train['price'], test_size=0.2)

    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    
    del X, X_train, X_test, y_train
    gc.collect()
    
    return preds, y_test

# Huấn Luyện

**Ridge Regression Model**

Regression Ridge là một cách để tạo mô hình phân tích khi số lượng biến dự báo trong một tập hợp vượt quá số lượng quan sát hoặc khi tập dữ liệu có đa cộng tuyến (tương quan giữa các biến dự báo)

Hồi quy OLS sử dụng công thức sau để ước tính các hệ số:



Nếu X là ma trận căn giữa và có tỷ lệ, thì ma trận sản phẩm chéo (X' X) gần như là số ít khi các cột X có tương quan cao. Hồi quy Ridge thêm một tham số ridge (k), của ma trận nhận dạng vào ma trận tích chéo, tạo thành một ma trận mới (X' X + kI). Công thức mới được sử dụng để tìm các hệ số:



In [ ]:
linear_model = Ridge(solver='lsqr', fit_intercept=False)

sparse_matrix_list = (X_train_name, X_train_brand, 
                      X_train_item_condition_id, X_train_shipping, 
                      X_train_main_cat, X_train_sub_cat, X_train_item_cat)

linear_preds, y_test = model_train_predict(model=linear_model, 
                                           matrix_list=sparse_matrix_list)

print('Item Description rmsle:', evaluate_orig_price(y_test, linear_preds))

sparse_matrix_list = (X_train_name, X_train_descp, X_train_brand, 
                      X_train_item_condition_id, X_train_shipping, 
                      X_train_main_cat, X_train_sub_cat, X_train_item_cat)


linear_preds, y_test = model_train_predict(model=linear_model, 
                                           matrix_list=sparse_matrix_list)
print('Item Description rmsle:', evaluate_orig_price(y_test, linear_preds))

# Kết quả

In [ ]:
sparse_matrix_list = (X_train_name, X_train_descp, X_train_brand, 
                      X_train_item_condition_id, X_train_shipping, 
                      X_train_main_cat, X_train_sub_cat, X_train_item_cat)

X_train = hstack(sparse_matrix_list).tocsr()
X_train

In [ ]:
sparse_matrix_list = (X_test_name, X_test_descp, X_test_brand, 
                      X_test_item_condition_id, X_test_shipping, 
                      X_test_main_cat, X_test_sub_cat, X_test_item_cat)
X_test = hstack(sparse_matrix_list).tocsr()

In [ ]:
y_train = train['price']
y_train

In [ ]:
linear_model.fit(X_train, y_train)

In [ ]:
preds = linear_model.predict(X_test)
preds

In [ ]:
preds = np.expm1(preds)
preds

In [ ]:
submission = pd.read_csv('sample_submission_stg2.csv')
submission

In [ ]:
submission.to_csv('submission.csv', index=False)